# How to run a semantic skills from file
Now that you're familiar with Kernel basics, let's see how the kernel allows you to run Semantic Skills and Semantic Functions stored on disk. 

A Semantic Skill is a collection of Semantic Functions, where each function is defined with natural language that can be provided with a text file. 

Refer to our [glossary](../../../GLOSSARY.md) for an in-depth guide to the terms.

The repository includes some examples under the [samples](https://github.com/microsoft/semantic-kernel/tree/main/samples) folder.

For instance, [this](../../skills/FunSkill/Joke/skprompt.txt) is the **Joke function** part of the **FunSkill skill**:

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE TOPIC BELOW.
JOKE MUST BE:
- G RATED
- WORKPLACE/FAMILY SAFE
NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY.
BE CREATIVE AND FUNNY. I WANT TO LAUGH.
+++++
{{$input}}
+++++
```

Note the special **`{{$input}}`** token, which is a variable that is automatically passed when invoking the function, commonly referred to as a "function parameter". 

We'll explore later how functions can accept multiple variables, as well as invoke other functions.


In the same folder you'll notice a second [config.json](../../skills/FunSkill/Joke/config.json) file. The file is optional, and is used to set some parameters for large language models like Temperature, TopP, Stop Sequences, etc.

```
{
  "schema": 1,
  "type": "completion",
  "description": "Generate a funny joke",
  "completion": {
    "max_tokens": 500,
    "temperature": 0.5,
    "top_p": 0.5
  }
}
```

Given a semantic function defined by these files, this is how to load and use a file based semantic function.

Load and configure the kernel, as usual, loading also the AI backend settings defined in the [Setup notebook](0-AI-settings.ipynb):

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.8.48.1-preview"
#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAICompletionBackend("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAICompletionBackend("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.8.48.1-preview

Import the skill and all its functions:

In [2]:
// note: using skills from the repo
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "..", "skills");

var skill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

How to use the skill functions, e.g. generate a joke about "*time travel to dinosaur age*":

In [3]:
var result = await kernel.RunAsync("time travel to dinosaur age", skill["Joke"]);

Console.WriteLine(result);


Possible jokes:

- I always wanted to time travel to the dinosaur age, until I realized I'm allergic to pollen. You know how much pollen there was back then? I sneezed so hard I caused the meteor strike.
- My friend said he could take me to the dinosaur age with his homemade time machine. I was skeptical, but he insisted it was safe and easy. He just needed a microwave, a banana peel, and a flux capacitor. I agreed to try it, but when I got there, I saw a huge T-rex staring at me. I turned to my friend and asked, "How do we get back?" He said, "Back? I don't know how to get back. I just wanted to see if it worked."
- I signed up for a time travel tour to the dinosaur age. It was supposed to be a once-in-a-lifetime experience. They gave me a suit, a helmet, and a tranquilizer gun. They said I could observe and photograph the dinosaurs, but not interfere with them. They also warned me not to step on anything, because it could alter the course of history. I was so excited, until I saw th

Great, now that you know how to load a skill from disk, let's show how you can [create and run a semantic function inline.](./3-semantic-function-inline.ipynb)